In [1]:
#import all the library
import numpy as np
import matplotlib.pyplot as plt
import cv2
import math
import time

In [44]:
class ComputeDisparity:
    #class attribute
    #1. left image
    #2. right image
    #3. ground true disparity map
    #4. compute true disparity map 
    #5. compute time
    
    def __init__(self, left_filepath, right_filepath, disparity_filepath):
        #import left image
        self.left_image = cv2.cvtColor(cv2.imread(left_filepath),cv2.COLOR_BGR2GRAY)
        #import right image 
        self.right_image = cv2.cvtColor(cv2.imread(right_filepath),cv2.COLOR_BGR2GRAY)
        #import the ground truth disparity map
        self.g_disparity_map = cv2.imread(disparity_filepath,-1)/256
        #create the variable to record the computed disparity map
        self.c_disparity_map = None
        #create the variable to record the computation time 
        self.com_time = 0

    def get_left(self):
        return self.left_image
    
    def get_right(self):
        return self.right_image
    
    def get_g_disparity_map(self):
        return self.g_disparity_map
    
    def get_c_disparity_map(self):
        return self.c_disparity_map
    
    def get_process_result(self):
        return self.process_result 
        
    def compute_ssd_disparity(self, window):
        start = time.time()
        #convert the images into numpy array
        left = np.asarray(self.left_image)
        right = np.asarray(self.right_image)
        #get the height and weight of the image
        height = self.right_image.shape[0]
        width = self.right_image.shape[1]
        
        #create the disparity map with all element equal to zero 
        disparity = np.zeros((width, height))
        disparity.shape = height, width
        #half of the window 
        window_half = int(window / 2)
        
        #for all the pixels in the left image
        for y in range(window_half, height - window_half):  
            print("\rProcessing.. %d%% complete"%(y / (height - window_half) * 100), end="", flush=True)
            for x in range(window_half, width - window_half):
                smallest_ssd = 65534
                smallest_pair_x = x
                
                #for all the pixels in the right image which is in the same height as the current pixels with the left image
                for right_w in range(window_half, width - window_half): 
                    #compute ssd for all the ssd on the pixels on the current height of right images 
                    current_ssd = 0
                    ssd_temp = 0                    
                    for v in range(-window_half, window_half+1):
                        for u in range(-window_half, window_half+1):
                            ssd_temp = left[y+v, x+u] - right[y+v,right_w+u] 
                            current_ssd += ssd_temp * ssd_temp
                        
                    #if the computed ssd is smaller than current ssd, update the result for this pixel
                    if current_ssd < smallest_ssd:
                        smallest_ssd  = current_ssd
                        smallest_pair_x = right_w

                #record the disparity value for current pixel
                disparity[y,x] =  abs(x-smallest_pair_x)
        
        #record the dispariry map 
        self.c_disparity_map = disparity
        #record the time
        end = time.time()
        self.com_time = end - start
        return self.c_disparity_map
        
    
    def compute_ssd_disparity_acc(self, window):
        start = time.time()
        #convert the images into numpy array
        left = np.asarray(self.left_image)
        right = np.asarray(self.right_image)
        #get the height and weight of the image  
        height = self.right_image.shape[0]
        width = self.right_image.shape[1]
        
        #create the disparity map with all element equal to zero 
        disparity = np.zeros((width, height))
        disparity.shape = height, width
        #half of the window 
        window_half = int(window / 2)
        for y in range(window_half, height - window_half):  
            print("\rProcessing.. %d%% complete"%(y / (height - window_half) * 100), end="", flush=True)
            for x in range(window_half, width - window_half):
                smallest_ssd = 65534
                smallest_pair_x = x
                #Compute the matrix in the left graph
                left_matrix = self.left_image[y-window_half:y+window_half,x-window_half:x+window_half]
                for right_w in range(window_half, width - window_half): 
                    #compute the right matrix 
                    right_matrix = self.right_image[y-window_half:y+window_half,right_w-window_half:right_w+window_half]
                    
                    #calculate the ssd with left and right matrix
                    current_ssd = np.sum(np.square(left_matrix-right_matrix))
                    
                    #record the smallest ssd by comparsion
                    if current_ssd < smallest_ssd:
                        smallest_ssd  = current_ssd
                        smallest_pair_x = right_w
                        
                #record the disparity for the current pixel
                disparity[y,x] =  np.absolute(x-smallest_pair_x)
            
        #record the disparity map
        self.c_disparity_map = disparity
        #record the computation time 
        end = time.time()
        self.com_time = end - start
        return self.c_disparity_map
    

    def compute_ssd_disparity_acc2(self, window, search_range):
        start = time.time()
        #convert the images into numpy array
        left = np.asarray(self.left_image)
        right = np.asarray(self.right_image)
        #get the height and weight of the image 
        height = self.right_image.shape[0]
        width = self.right_image.shape[1]
        
        #create the disparity map with all element equal to zero 
        disparity = np.zeros((width, height))
        disparity.shape = height, width
        
        #half of the window 
        window_half = int(window / 2)
        for y in range(window_half, height - window_half):  
            print("\rProcessing.. %d%% complete"%(y / (height - window_half) * 100), end="", flush=True)
            for x in range(window_half, width - window_half):
                smallest_ssd = 65534
                smallest_pair_x = x
                
                #calculate the search range for the given value
                start = window_half
                if x-search_range<window_half:start = window_half
                else: start = x-search_range
                end = width - window_half
                if x + search_range > width - window_half: end = width - window_half
                else: end = x + int(search_range/2)
                
                #for all the pixels in the right image which is in the same height as the current pixels with the left image
                for right_w in range(start, end): 
                    current_ssd = 0
                    ssd_temp = 0                    
                    for v in range(-window_half, window_half+1):
                        for u in range(-window_half, window_half+1):
                            ssd_temp = left[y+v, x+u] - right[y+v,right_w+u]
                            current_ssd += ssd_temp * ssd_temp
                            
                    #record the smallest ssd by comparsion
                    if current_ssd < smallest_ssd:
                        smallest_ssd  = current_ssd
                        smallest_pair_x = right_w
                        
                #record the disparity value for this pixel
                disparity[y,x] = np.absolute(smallest_pair_x-x)
            
        #record the disparity map
        self.c_disparity_map = disparity
        #record the computation time
        end = time.time()
        self.com_time = end - start
        return self.c_disparity_map


    def compute_sad_disparity(self):
        start = time.time()
        ##StereoBM uses SAD
        #copy the code from workshop 
        stereoBM = cv2.StereoBM_create(numDisparities = 32, blockSize =21)
        disparityBM = stereoBM.compute(self.left_image, self.right_image)
        self.c_disparity_map = disparityBM/16
        #record the computation time 
        end = time.time()
        self.com_time = end - start
        return disparityBM
    
    def compute_sgbm_disparity(self):
        start = time.time()
        ##stereoSGBM uses SGBM
        #copy the code from workshop 
        stereoSGBM = cv2.StereoSGBM_create(numDisparities = 32,blockSize = 21)
        disparitySGBM = stereoSGBM.compute(self.left_image, self.right_image)
        self.c_disparity_map = disparitySGBM/16
        #record the computation time 
        end = time.time()
        self.com_time = end - start
        return disparitySGBM
    
    def compute_zssd_disparity(self, kernel, search_range):
        #apply guassian filter
        sigma = 0.3 * ((kernel - 1) * 0.5 - 1) + 0.8
        self.left_image = cv2.GaussianBlur(self.left_image,(kernel,kernel),sigma, cv2.BORDER_DEFAULT)
        self.right_image = cv2.GaussianBlur(self.right_image,(kernel,kernel),sigma, cv2.BORDER_DEFAULT)
        
        start = time.time()
        left = np.asarray(self.left_image)
        right = np.asarray(self.right_image)    
        h = self.right_image.shape[0]
        w = self.right_image.shape[1]

        disparity = np.zeros((w, h), np.uint8)
        disparity.shape = h, w
        
        kernel_half = int(kernel / 2)
        for y in range(kernel_half, h - kernel_half):  
            print("\rProcessing.. %d%% complete"%(y / (h - kernel_half) * 100), end="", flush=True)
            for x in range(kernel_half, w - kernel_half):
                smallest_ssd = 65534
                smallest_pair_x = x
                
                start = kernel_half
                if x-search_range<kernel_half:start = kernel_half
                else: start = x-search_range
                
                end = w - kernel_half
                if x + search_range > w - kernel_half: end = w - kernel_half
                else: end = x + int(search_range/2)
                for right_w in range(start, end): 
                    ssd = 0
                    ssd_temp = 0    
                    left_mean = 0
                    right_mean = 0
                    left_sum = 0
                    right_sum = 0
                    # get window mean
                    for i in range(-kernel_half, kernel_half+1):
                        for j in range(-kernel_half, kernel_half+1):
                            left_sum += left[y+i, x+j]
                            right_sum += right[y+i,right_w+j]
                            
                    left_mean = left_sum/(kernel**2)
                    right_mean = right_sum/(kernel**2)
                    
                    for v in range(-kernel_half, kernel_half+1):
                        for u in range(-kernel_half, kernel_half+1):
                            ssd_temp = (left[y+v, x+u]-left_mean) - (right[y+v,right_w+u]-right_mean)
                            ssd += ssd_temp * ssd_temp

                    if ssd < smallest_ssd:
                        smallest_ssd  = ssd
                        smallest_pair_x = right_w
                
                disparity[y,x] =  abs(smallest_pair_x-x)

        self.c_disparity_map = disparity
        end = time.time()
        self.com_time = end - start
        return disparity


    def compute_ncc_disparity_acc(self, window,search_range):
        start_time = time.time()
        #convert the images into numpy array
        left = np.asarray(self.left_image)
        right = np.asarray(self.right_image)  
        #get the height and weight of the image 
        height = self.right_image.shape[0]
        width = self.right_image.shape[1]
        
        #create the disparity map with all element equal to zero 
        disparity= np.zeros((width, height), np.uint8)
        disparity.shape = height, width
        
        #half of the window 
        window_half = int(window/ 2)
        
        for y in range(window_half, height - window_half):              
            print("\rProcessing.. %d%% complete"%(y / (height - window_half) * 100), end="", flush=True)
            for x in range(window_half, width - window_half):
                max_ncc = 0
                max_pair_x = x
                
                #calculate the search range for the given value
                start = window_half
                if x-search_range<window_half:start = window_half
                else: start = x-search_range
                
                end = width - window_half
                if x + search_range > width - window_half: end = width - window_half
                else: end = x + int(search_range/2)
                    
                #for all the pixels in the right image which is in the same height as the current pixels with the left image
                for right_w in range(start, end): 
                    #the window values on the left image
                    window_left = np.zeros((window, window), np.uint8)
                    #the window value on the right image
                    window_right = np.zeros((window, window), np.uint8)
                    current_ncc = 0
                    ncc_temp = 0                            
                    for v in range(-window_half, window_half+1):
                        for u in range(-window_half, window_half+1):
                            #fill in the value into the window on the left
                            window_left[v+window_half, u+window_half] = left[y+v, x+u]
                            #fill in the value into the window on the right 
                            window_right[v+window_half, u+window_half] = right[y+v, right_w+u]
                            ncc_temp = left[y+v, x+u] * right[y+v, right_w+u]
                            current_ncc += ncc_temp 
                    
                    current_ncc = current_ncc/(np.linalg.norm(window_left)*np.linalg.norm(window_right))
                    #record the larger ncc by comparsion
                    if current_ncc > max_ncc:
                        max_ncc = current_ncc
                        max_pair_x = right_w
                        
                #record the disparity value for the current pixel       
                disparity[y,x] = abs(x-max_pair_x)
        
        #record the disparity map
        self.c_disparity_map = disparity
        #record the computation time
        end_time = time.time()
        self.com_time = end_time - start_time 
        return self.c_disparity_map

            
    def compute_ssd_disparity_withGua(self, window, search_range):
        #apply guassian filter
        sigma = 0.3 * ((window - 1) * 0.5 - 1) + 0.8
        self.left_image = cv2.GaussianBlur(self.left_image,(window,window),sigma, cv2.BORDER_DEFAULT)
        self.right_image = cv2.GaussianBlur(self.right_image,(window,window),sigma, cv2.BORDER_DEFAULT)
        
        #convert the images into numpy array
        left = np.asarray(self.left_image)
        right = np.asarray(self.right_image)
        #get the height and weight of the image 
        height = self.right_image.shape[0]
        width = self.right_image.shape[1]
        
        #create the disparity map with all element equal to zero 
        disparity = np.zeros((width, height))
        disparity.shape = height, width
        
        #half of the window 
        window_half = int(window / 2)
        for y in range(window_half, height - window_half):  
            print("\rProcessing.. %d%% complete"%(y / (height - window_half) * 100), end="", flush=True)
            for x in range(window_half, width - window_half):
                smallest_ssd = 65534
                smallest_pair_x = x
                
                #calculate the search range for the given value
                start = window_half
                if x-search_range<window_half:start = window_half
                else: start = x-search_range
                end = width - window_half
                if x + search_range > width - window_half: end = width - window_half
                else: end = x + int(search_range/2)
                
                #for all the pixels in the right image which is in the same height as the current pixels with the left image
                for right_w in range(start, end): 
                    current_ssd = 0
                    ssd_temp = 0                    
                    for v in range(-window_half, window_half+1):
                        for u in range(-window_half, window_half+1):
                            ssd_temp = left[y+v, x+u] - right[y+v,right_w+u]
                            current_ssd += ssd_temp * ssd_temp
                            
                    #record the smallest ssd by comparsion
                    if current_ssd < smallest_ssd:
                        smallest_ssd  = current_ssd
                        smallest_pair_x = right_w
                        
                #record the disparity value for this pixel
                disparity[y,x] = np.absolute(smallest_pair_x-x)
            
        #record the disparity map
        self.c_disparity_map = disparity
        #record the computation time
        end = time.time()
        self.com_time = end - start
        return self.c_disparity_map

    def compute_ssd_disparity_smooth(self, kernel, search_range, disparityMap, weight):
        #apply guassian filter
        sigma = 0.3 * ((kernel - 1) * 0.5 - 1) + 0.8
        self.left_image = cv2.GaussianBlur(self.left_image,(kernel,kernel),sigma, cv2.BORDER_DEFAULT)
        self.right_image = cv2.GaussianBlur(self.right_image,(kernel,kernel),sigma, cv2.BORDER_DEFAULT)
        
        start = time.time()
        left = np.asarray(self.left_image)
        right = np.asarray(self.right_image)    
        h = self.right_image.shape[0]
        w = self.right_image.shape[1]

        depth = np.zeros((w, h), np.uint8)
        depth.shape = h, w
        
        kernel_half = int(kernel / 2)
        for y in range(kernel_half, h - kernel_half):  
            print("\rSmoothing.. %d%% complete"%(y / (h - kernel_half) * 100), end="", flush=True)
            for x in range(kernel_half, w - kernel_half):
                smallest_ssd = 65534
                smallest_pair_x = x
                
                start = kernel_half
                if x-search_range<kernel_half:start = kernel_half
                else: start = x-search_range
                
                end = w - kernel_half
                if x + search_range > w - kernel_half: end = w - kernel_half
                else: end = x + int(search_range/2)
                for right_w in range(start, end): 
                    ssd = 0
                    ssd_temp = 0            
                    
                    distance = np.abs(x - right_w)
                    smooth = weight * (np.abs(distance - disparityMap[y-1, x]) +
                                       np.abs(distance - disparityMap[y, x-1]) +
                                       np.abs(distance - disparityMap[y+1, x]) +
                                       np.abs(distance - disparityMap[y, x+1]))
                    for v in range(-kernel_half, kernel_half+1):
                        for u in range(-kernel_half, kernel_half+1):
                            ssd_temp = left[y+v, x+u] - right[y+v,right_w+u]
                            ssd += ssd_temp * ssd_temp

                    ssd = ssd + smooth        
                    if ssd < smallest_ssd:
                        smallest_ssd  = ssd
                        smallest_pair_x = right_w
                
                depth[y,x] =  abs(smallest_pair_x-x)

        self.c_disparity_map = depth
        end = time.time()
        self.com_time = end - start
       

    def cal_rms(self):
        #calculate the rms
        sum_of_sub = 0
        num_valid = 0
        for i in range(len(self.g_disparity_map)):
            for j in range(len(self.g_disparity_map[0])):
                if(self.g_disparity_map[i][j]!=0):           
                    sum_of_sub = sum_of_sub+((self.g_disparity_map[i][j]-self.c_disparity_map[i][j])**2)
                    num_valid = num_valid+1
        mse = sum_of_sub/num_valid
        rmse = math.sqrt(mse)
        return rmse
    
    def fra_pixel_dif(self):
        #cound the number of err in different range
        err = 0
        num_valid = 0
        num_lessfour = 0
        num_lesstwo = 0
        num_lessone = 0
        num_lesspoint5 = 0
        num_lesspoint25 = 0
        for i in range(len(self.g_disparity_map)):
            for j in range(len(self.g_disparity_map[0])):
                if(self.g_disparity_map[i][j]!=0):           
                    err = abs(self.g_disparity_map[i][j]-self.c_disparity_map[i][j])
                    num_valid = num_valid+1
                    if err<4: 
                        num_lessfour = num_lessfour+1
                    if err<2:
                        num_lesstwo = num_lesstwo+1
                    if err<1: 
                        num_lessone = num_lessone+1
                    if err<0.5:
                        num_lesspoint5 = num_lesspoint5+1
                    if err<0.25:
                        num_lesspoint25 = num_lesspoint25+1
                    
        return num_lessfour/num_valid, num_lesstwo/num_valid,num_lessone/num_valid, num_lesspoint5/num_valid,num_lesspoint25/num_valid
        
    def analysis(self):
        #calculate the rms error betweem the value in your disparity map and those in the ground truth. 
        four, two, one, point5, point25 = self.fra_pixel_dif()
        print("the root mean squre is: "+str(self.cal_rms()))
        print("The fractions of pixels with the error less than 4 is: "+str(four))
        print("The fractions of pixels with the error less than 2 is: "+str(two))
        print("The fractions of pixels with the error less than 1 is: "+str(one))
        print("The fractions of pixels with the error less than 0.5 is: "+str(point5))
        print("The fractions of pixels with the error less than 0.25 is: "+str(point25))
        print("The computation time is:"+str(self.com_time))
    
    
    #copy from workshop2 worksheet COMP90086 Computer Vision
    #option=0 is the right image
    #option=1 is the left image
    #option=2 is self computed disparity map
    #option=3 is ground true disparity map
    def imshow(self, option,  *args, **kwargs):
        if option==0:
                image = self.left_image
        elif option==1:
                image = self.right_image
        elif option==2:
                image = self.c_disparity_map
        elif option==3:
                image = self.g_disparity_map
        else:
                print("There is no such option")
                return
            
        if len(image.shape) == 3:
          # Height, width, channels
          # Assume BGR, do a conversion since 
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        else:
          # Height, width - must be grayscale
          # convert to RGB, since matplotlib will plot in a weird colormap (instead of black = 0, white = 1)
          image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        # Draw the image
        plt.imshow(image, *args, **kwargs)
        # We'll also disable drawing the axes and tick marks in the plot, since it's actually an image
        plt.axis('off')
        # Make sure it outputs
        plt.show()


## The result of using openCV stereoBM(SAD) function

In [5]:
#import the image with its filepath
sad_result = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
sad_result.compute_sad_disparity()
sad_result.analysis()

the root mean squre is: 29.59693045192903
The fractions of pixels with the error less than 4 is: 0.4118821540800236
The fractions of pixels with the error less than 2 is: 0.37002121408070426
The fractions of pixels with the error less than 1 is: 0.3134238618702424
The fractions of pixels with the error less than 0.5 is: 0.24281614085242034
The fractions of pixels with the error less than 0.25 is: 0.15079013942302238
The computation time is:0.004376888275146484


## The result of using openCV StereoSGBM function

In [6]:
#import the image with its filepath
sgbd_result = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
sgbd_result.compute_sgbm_disparity()
sgbd_result.analysis()

the root mean squre is: 22.34015359959334
The fractions of pixels with the error less than 4 is: 0.5834439415081283
The fractions of pixels with the error less than 2 is: 0.4917582729242532
The fractions of pixels with the error less than 1 is: 0.38864876515899216
The fractions of pixels with the error less than 0.5 is: 0.2792544441797411
The fractions of pixels with the error less than 0.25 is: 0.16809039240377088
The computation time is:0.04082894325256348


# Accelerated the algorithm

## The result of using unaccelerated SSD function with window size 3

In [8]:
#import the image with its filepath
ssd_result3 = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
ssd_result3.compute_ssd_disparity(3)

Processing.. 0% complete

/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/1296903645.py:82: RuntimeWarning: overflow encountered in ubyte_scalars
  ssd_temp = left[y+v, x+u] - right[y+v,right_w+u]
/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/1296903645.py:83: RuntimeWarning: overflow encountered in ubyte_scalars
  ssd += ssd_temp * ssd_temp


Processing.. 99% complete

In [9]:
ssd_result3.analysis()

the root mean squre is: 84.13781633006359
The fractions of pixels with the error less than 4 is: 0.24247580800689741
The fractions of pixels with the error less than 2 is: 0.18640029949290407
The fractions of pixels with the error less than 1 is: 0.1268307070982087
The fractions of pixels with the error less than 0.5 is: 0.07507742572235647
The fractions of pixels with the error less than 0.25 is: 0.04089666360367106
The computation time is:2387.2629749774933


## The result of using accelerated(by using matrix attempt) SSD function with window size 3

In [11]:
#import the image with its filepath
ssd_acc_result3 = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
ssd_acc_result3.compute_ssd_disparity_acc(3)
ssd_acc_result3.analysis()


the root mean squre is: 94.85333661919
The fractions of pixels with the error less than 4 is: 0.2323333388913156
The fractions of pixels with the error less than 2 is: 0.11979716234556895
The fractions of pixels with the error less than 1 is: 0.09134234324356885
The fractions of pixels with the error less than 0.5 is: 0.060987972345824
The fractions of pixels with the error less than 0.25 is: 0.0268746735623458835634
The computation time is:4577.931536912918



## The result of using accelerated(by restricting search area attempt) SSD function with window size 3

In [23]:
#import the image with its filepath
ssd_acc2_result3 = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
ssd_acc2_result3.compute_ssd_disparity_acc2(3,70)
ssd_acc2_result3.analysis()

Processing.. 0% complete

/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/1101813449.py:161: RuntimeWarning: overflow encountered in ubyte_scalars
  ssd_temp = left[y+v, x+u] - right[y+v,right_w+u]
/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/1101813449.py:162: RuntimeWarning: overflow encountered in ubyte_scalars
  current_ssd += ssd_temp * ssd_temp


Processing.. 99% completethe root mean squre is: 19.651625551372756
The fractions of pixels with the error less than 4 is: 0.37282328784217633
The fractions of pixels with the error less than 2 is: 0.28125106354014223
The fractions of pixels with the error less than 1 is: 0.19079059320014974
The fractions of pixels with the error less than 0.5 is: 0.11266151629627108
The fractions of pixels with the error less than 0.25 is: 0.06124856776594176
The computation time is:1666442368.770935


# Result of Different Window size

## The result of using accelerated(by restricting search area attempt) SSD function with window size 11

In [32]:
#import the image with its filepath
ssd_acc2_result11 = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
ssd_acc2_result11.compute_ssd_disparity_acc2(11,70)
ssd_acc2_result11.analysis()

Processing.. 1% complete

/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/1101813449.py:161: RuntimeWarning: overflow encountered in ubyte_scalars
  ssd_temp = left[y+v, x+u] - right[y+v,right_w+u]
/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/1101813449.py:162: RuntimeWarning: overflow encountered in ubyte_scalars
  current_ssd += ssd_temp * ssd_temp


Processing.. 99% completethe root mean squre is: 16.870386424972235
The fractions of pixels with the error less than 4 is: 0.5812204335840452
The fractions of pixels with the error less than 2 is: 0.4993136620948621
The fractions of pixels with the error less than 1 is: 0.37478587391802515
The fractions of pixels with the error less than 0.5 is: 0.23419437543250632
The fractions of pixels with the error less than 0.25 is: 0.1304268908325676
The computation time is:1666445416.38239


## The result of using accelerated(by restricting search area attempt) SSD function with window size 15

In [36]:
#import the image with its filepath
ssd_acc2_result15 = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
ssd_acc2_result15.compute_ssd_disparity_acc2(15,70)
ssd_acc2_result15.analysis()

Processing.. 1% complete

/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/409812648.py:161: RuntimeWarning: overflow encountered in ubyte_scalars
  ssd_temp = left[y+v, x+u] - right[y+v,right_w+u]
/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/409812648.py:162: RuntimeWarning: overflow encountered in ubyte_scalars
  current_ssd += ssd_temp * ssd_temp


Processing.. 99% completethe root mean squre is: 16.676194032016095
The fractions of pixels with the error less than 4 is: 0.6185436023097256
The fractions of pixels with the error less than 2 is: 0.5236814938343033
The fractions of pixels with the error less than 1 is: 0.3879907883243145
The fractions of pixels with the error less than 0.5 is: 0.2408422103483874
The fractions of pixels with the error less than 0.25 is: 0.13456760711976312
The computation time is:1666453118.4759521


# Result of different Algorithm 

## The result of using accelerated(by restricting search area attempt) NCC function with window size 3

In [38]:
#import the image with its filepath
image_acc_ncc3 = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
dis_map = image_acc_ncc3.compute_ncc_disparity_acc(3,70)
image_acc_ncc3.analysis()


Processing.. 99% completethe root mean squre is: 24.720734627300093
The fractions of pixels with the error less than 4 is: 0.1282033829084845
The fractions of pixels with the error less than 2 is: 0.06368761982552269
The fractions of pixels with the error less than 1 is: 0.03235430918104573
The fractions of pixels with the error less than 0.5 is: 0.015394389045820146
The fractions of pixels with the error less than 0.25 is: 0.007464633745136076
The computation time is:568.5074129104614



## The result of using accelerated(by restricting search area attempt) NCC function with window size 7

In [35]:
#import the image with its filepath
image_acc_ncc7 = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
dis_map = image_acc_ncc7.compute_ncc_disparity_acc(7,70)
#image1.compute_sgbm_disparity()
image_acc_ncc7.analysis()

Processing.. 0% complete

/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/409812648.py:310: RuntimeWarning: overflow encountered in ubyte_scalars
  ncc_temp = left[y+v, x+u] * right[y+v, right_w+u]


Processing.. 99% completethe root mean squre is: 25.372860719871486
The fractions of pixels with the error less than 4 is: 0.12447106603591646
The fractions of pixels with the error less than 2 is: 0.06170234489330565
The fractions of pixels with the error less than 1 is: 0.0320593540482592
The fractions of pixels with the error less than 0.5 is: 0.014838512064799374
The fractions of pixels with the error less than 0.25 is: 0.006920101192299402
The computation time is:2020.4963929653168


## The result of using accelerated(by restricting search area attempt) ZSSD function with window size 3

In [25]:
#import the image with its filepath
image_zssd3 = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
image_zssd_map = image_zssd3.compute_zssd_disparity(3,70)
#image1.compute_sgbm_disparity()
image_zssd3.analysis()

Processing.. 99% completethe root mean squre is: 21.176494069533444
The fractions of pixels with the error less than 4 is: 0.37430940793429307
The fractions of pixels with the error less than 2 is: 0.2976551066943471
The fractions of pixels with the error less than 1 is: 0.2078526131890322
The fractions of pixels with the error less than 0.5 is: 0.1230416680847202
The fractions of pixels with the error less than 0.25 is: 0.06599053874689446
The computation time is:1666330300.1154158


## The result of using accelerated(by restricting search area attempt) ZSSD function with window size 15

In [47]:
#import the image with its filepath
image_zssd15 = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
image_zssd_map = image_zssd15.compute_zssd_disparity(15,70)
#image1.compute_sgbm_disparity()
image_zssd15.analysis()

Processing.. 1% complete

KeyboardInterrupt: 

# Guassian Blur

## The result of using accelerated(by restricting search area attempt) SSD function with window size 3 with Guassian kernel

In [14]:
#import the image with its filepath
ssd_gua = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
ssd_gua_map = ssd_gua.compute_ssd_disparity_withGua(3,70)
#image1.compute_sgbm_disparity()
ssd_gua.analysis()


Processing.. 0% complete

C:\Users\Sibo\.conda\envs\CV\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: overflow encountered in ubyte_scalars
C:\Users\Sibo\.conda\envs\CV\lib\site-packages\ipykernel_launcher.py:134: RuntimeWarning: overflow encountered in ubyte_scalars


Processing.. 99% completethe root mean squre is: 19.007878041932518
The fractions of pixels with the error less than 4 is: 0.41073636683342973
The fractions of pixels with the error less than 2 is: 0.31683853475365575
The fractions of pixels with the error less than 1 is: 0.2144096926794405
The fractions of pixels with the error less than 0.5 is: 0.12503828744512133
The fractions of pixels with the error less than 0.25 is: 0.06665986000975621
The computation time is:1666326710.2070978


## The result of using accelerated(by restricting search area attempt) SSD function with window size 15 with Guassian kernel

In [45]:
#import the image with its filepath
ssd_gua15 = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
ssd_gua15_map = ssd_gua15.compute_ssd_disparity_withGua(15,70)
#image1.compute_sgbm_disparity()
ssd_gua15.analysis()

Processing.. 1% complete

/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/3875852866.py:372: RuntimeWarning: overflow encountered in ubyte_scalars
  ssd_temp = left[y+v, x+u] - right[y+v,right_w+u]
/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/3875852866.py:373: RuntimeWarning: overflow encountered in ubyte_scalars
  current_ssd += ssd_temp * ssd_temp


Processing.. 99% completethe root mean squre is: 16.210511741329473
The fractions of pixels with the error less than 4 is: 0.6197801449817922
The fractions of pixels with the error less than 2 is: 0.4807428331574947
The fractions of pixels with the error less than 1 is: 0.2867077334966931
The fractions of pixels with the error less than 0.5 is: 0.15086955042031105
The fractions of pixels with the error less than 0.25 is: 0.07828789889845603
The computation time is:1666465579.1001868


# Smooth

## The result of using accelerated(by restricting search area attempt) SSD function with window size 3 with Guassian kernel and Smooth map 

In [37]:
#import the image with its filepath
image_3smooth = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
image_3smooth_map = image_3smooth.compute_ssd_disparity_acc2(3,70)
image_3smooth_map_2 = image_3smooth.compute_ssd_disparity_smooth(3, 70, image_3smooth_map, 0.5)
image_3smooth.analysis()


Processing.. 99% completethe root mean squre is: 18.126510383655095
The fractions of pixels with the error less than 4 is: 0.4407991015212878
The fractions of pixels with the error less than 2 is: 0.34227274274240205
The fractions of pixels with the error less than 1 is: 0.22806838421309375
The fractions of pixels with the error less than 0.5 is: 0.1311189009517975
The fractions of pixels with the error less than 0.25 is: 0.06962075576580562
The computation time is:1666442064.4756658



## The result of using accelerated(by restricting search area attempt) SSD function with window size 11 with Guassian kernel and Smooth map 

In [40]:
#import the image with its filepath
image_smooth = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
image_smooth_map = image_smooth.compute_ssd_disparity_acc2(11,70)
image_smooth_map_2 = image_smooth.compute_ssd_disparity_smooth(11, 70, image_smooth_map, 0.5)
#image1.compute_sgbm_disparity()
image_smooth.analysis()

Processing.. 1% complete

/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/353087008.py:163: RuntimeWarning: overflow encountered in ubyte_scalars
  ssd_temp = left[y+v, x+u] - right[y+v,right_w+u]
/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/353087008.py:164: RuntimeWarning: overflow encountered in ubyte_scalars
  current_ssd += ssd_temp * ssd_temp


Smoothing.. 1% completete

/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/353087008.py:430: RuntimeWarning: overflow encountered in ubyte_scalars
  ssd_temp = left[y+v, x+u] - right[y+v,right_w+u]
/var/folders/fm/nscr_s1n601dsyhjhm39__th0000gn/T/ipykernel_1728/353087008.py:431: RuntimeWarning: overflow encountered in ubyte_scalars
  ssd += ssd_temp * ssd_temp


Smoothing.. 99% completethe root mean squre is: 16.500493281377974
The fractions of pixels with the error less than 4 is: 0.5888665781801268
The fractions of pixels with the error less than 2 is: 0.48090165515207206
The fractions of pixels with the error less than 1 is: 0.3064130052524702
The fractions of pixels with the error less than 0.5 is: 0.16571940691329454
The fractions of pixels with the error less than 0.25 is: 0.08501514481162577
The computation time is:1666459601.738912


## The result of using accelerated(by restricting search area attempt) SSD function with window size 15 with Guassian kernel and Smooth map 

In [46]:
#import the image with its filepath
image_smooth15 = ComputeDisparity("./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-left.jpg",
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-right.jpg", 
                         "./Dataset/2018-07-09-16-11-56_2018-07-09-16-11-56-702-disparity.png")
image_smooth_map15 = image_smooth15.compute_ssd_disparity_acc2(15,70)
image_smooth_map15_2 = image_smooth15.compute_ssd_disparity_smooth(15, 70, image_smooth_map, 0.5)
image_smooth15.analysis()


Smoothing.. 99% completethe root mean squre is: 16.21689056345499
The fractions of pixels with the error less than 4 is: 0.6213797093557499
The fractions of pixels with the error less than 2 is: 0.4833066739271007
The fractions of pixels with the error less than 1 is: 0.2883980532961236
The fractions of pixels with the error less than 0.5 is: 0.15194727109780032
The fractions of pixels with the error less than 0.25 is: 0.07863957617216304
The computation time is:1666465459.183312

